In [14]:
import kenlm 
import nltk 
import codecs
from collections import Counter
from itertools import combinations, permutations, product

inputwords_file = "./test_set_public"
outputwords_file = "./output_words.txt"

## 将文件导入到 kenlm 语言模型
from itertools import combinations, permutations
model = kenlm.LanguageModel("./lm.bin")

## 读取文件
def readfile(filepath):
    fp = codecs.open(filepath, "r", encoding="utf-8")
    content = fp.read()
    fp.close()
    return content

## 保存文件
def savefile(savepath, content):
    # 如果文件已经存在，则在文件的末尾继续添加
    fp = codecs.open(savepath,"a",encoding='utf-8')
    fp.write(content)
    fp.close()
    
## 按行加载文件
def readwordslist(filepath):
    wordslist = readfile(filepath).splitlines()
    return wordslist

## 初始化写入文件
def start_savefile(savepath):
    fp = codecs.open(savepath,"w",encoding='utf-8')
    fp.write("")
    fp.close()
    
## 对 a 和 an 分布的可能性进行枚举，然后对句子中 a/an 所在的位置进行替换插入
def generate(template_str, count):
    res = []
    tem_arr =  template_str.split()
    all_iters = list(product(("a", "an"), repeat = count)) 
    for iter in all_iters:
        sentence = get_one_sentence(tem_arr, iter)
        res.append(sentence)
    return res

## 将列表中的数据插入到句子的占位符中
def get_one_sentence(tem_arr, iter):
    s = []
    id = 0
    for i in range(0,len(tem_arr)):
        term = tem_arr[i]
        if term != "=":
            s.append(term)
        else:
            s.append(iter[id])
            id += 1
    return ' '.join(s)         


changed_line_num = 0

start_savefile(outputwords_file)

## 判断句子中是否存在一个 a/an ，如果有就将对应的 a 替换成 an
# 分别对含有 a 和 an 的句子进行打分，用语言模型判别每个句子的的得分
# 如果替换后的得分更加高了，那么说明原来句子里的 a/an 使用错误。
for line in readwordslist(inputwords_file):   
    if " a " in line or " an " in line:
        ## 获取句子中含有的 a/an 单独子串的数量
        count = Counter(nltk.word_tokenize(line))["a"] + Counter(nltk.word_tokenize(line))["an"]
        ## 将句子中相应位置的子串都变为占位符，便于进行修改
        # 占位符可以任意选择，这里选择等号，是因为常用的美元符号在我们的测试集数据中已经包含了。
        line_new = line.replace(" a ", " = ")
        line_new = line.replace(" an ", " = ")
        ## 得到新生成的替换后的句子列表
        candidates = generate(line_new, count)
        
        ## 判断得分最高的句子是否为原句子
        line_best = line 
        changed = 0     # 相比较使用句子字符串比对或者重新比较原句子和最高分句子的得分，使用标志位的方法更加方便。
        for s in candidates:
            if model.score(s) > model.score(line_best):
                line_best = s 
                changed += 1               
        if changed != 0:     
            changed_line_num += 1
            str_output = str(changed_line_num) + ":\n" + line + "\n>>\n" + line_best + "\n\n"
            savefile(outputwords_file, str_output)
            # print str_output
                       
str_output = "The total number of sentences that need to be corrected is : " + str(changed_line_num)
print str_output

The total number of sentences that need to be corrected is : 42455
